# Морфологічний аналіз

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
# Jupyter rendering stuff
from IPython.display import display, Math, Latex, HTML, clear_output

In [11]:
import itertools

## Input data generation

In [2]:
def generate_probabilities(*sizes):
    result = [np.random.random(size) for size in sizes]
    for vec in result:
        vec /= np.sum(vec)
    return result

In [9]:
def generate_cross_relation(size1, size2):
    return np.random.random(size=(size1, size2)) * 2 - 1

In [109]:
outer_f_s = (4,2,4,3,1,2,3)
outer_f_p = generate_probabilities(*outer_f_s)
inner_f_s = (5,4)
inner_f_p = generate_probabilities(*inner_f_s)

In [113]:
outer_cross_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.combinations(enumerate(outer_f_s), 2)}
inner_cross_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.combinations(enumerate(inner_f_s), 2)}
out_in_cross_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.product(enumerate(outer_f_s), enumerate(inner_f_s))}

## Calculating table for outer factors

In [115]:
def calculate_outer_correlation(factors):
    global outer_cross_rel
    result = 1
    for (f1,a1), (f2, a2) in itertools.combinations(enumerate(factors), 2):
        result *= outer_cross_rel[f1, f2][a1, a2] + 1
    return result

In [152]:
df_outer = pd.DataFrame(index=pd.MultiIndex.from_product(tuple(range(size) for size in outer_f_s)))

In [162]:
df_outer['C'] = calculate_outer_correlation(label for label in df_outer.index.labels)

In [163]:
df_outer

C
0 0 0 0 0 0 0  1.964470e-03
            1  4.888726e-04
            2  4.564238e-03
          1 0  1.601875e-02
            1  8.035971e-05
            2  4.574260e-03
      1 0 0 0  3.693722e-03
            1  5.254163e-04
            2  7.600397e-05
          1 0  2.226217e-01
            1  6.383609e-04
            2  5.630007e-04
      2 0 0 0  1.215587e-03
            1  7.401839e-04
            2  1.111851e-03
          1 0  3.826477e-03
            1  4.696909e-05
            2  4.301589e-04
    1 0 0 0 0  2.504350e-05
            1  6.436912e-05
            2  1.274180e-05
          1 0  4.316307e-04
            1  2.236425e-05
            2  2.699090e-05
      1 0 0 0  5.415841e-05
            1  7.956783e-05
            2  2.440343e-07
          1 0  6.899273e-03
            1  2.043311e-04
            2  3.820828e-06
...                     ...
3 1 2 1 0 0 0  5.080988e-04
            1  7.515075e-04
            2  6.013794e-05
          1 0  2.792606e-01
            1  8.326341e-03
            2  4.062368e-03
      2 0 0 0  3.702979e-09
            1  2.344502e-08
            2  1.948228e-08
          1 0  1.062973e-07
            1  1.356692e-08
            2  6.873538e-08
    3 0 0 0 0  3.007654e-05
            1  2.053050e-04
            2  3.437813e-04
          1 0  1.777889e-03
            1  2.446447e-04
            2  2.497631e-03
      1 0 0 0  8.251259e-04
            1  3.219444e-03
            2  8.352641e-05
          1 0  3.605099e-01
            1  2.835550e-02
            2  4.485285e-03
      2 0 0 0  4.678843e-06
            1  7.814719e-05
            2  2.105380e-05
          1 0  1.067690e-04
            1  3.594839e-05
            2  5.904816e-05

[576 rows x 1 columns]